In [ ]:
import torch
from diffusers import BitsAndBytesConfig, SD3Transformer2DModel
from diffusers import StableDiffusion3Pipeline
from huggingface_hub import login

# This will prompt an interactive login prompt in the console or display 
# a widget in a notebook where you can paste your token.
login("")

In [3]:

model_id = "stabilityai/stable-diffusion-3.5-large"

nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)




In [4]:
model_nf4 = SD3Transformer2DModel.from_pretrained(
    model_id,
    subfolder="transformer",
    quantization_config=nf4_config,
    torch_dtype=torch.bfloat16
)

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

: 

In [ ]:

pipeline = StableDiffusion3Pipeline.from_pretrained(
    model_id, 
    transformer=model_nf4,
    torch_dtype=torch.bfloat16
)
pipeline.enable_model_cpu_offload()

In [ ]:

device = "cuda"
pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", dtype=torch.bfloat16, device_map="cuda")
pipe.load_ip_adapter(
  "h94/IP-Adapter",
  subfolder="sdxl_models",
  weight_name="ip-adapter_sdxl.bin"
)
pipe.set_ip_adapter_scale(0.8)


In [ ]:
local_image_path = "C:/Users/majop/Downloads/rubeus-hagrid-1140-wikithumb.png" 
image = load_image("https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTAs_TDUTeHiZQ1tqLJlvItaBOjcmRTeoSbHw&s")

# Load the image from the file path, convert to RGB, and resize.
init_image = Image.open(local_image_path).convert("RGB")
init_image = init_image.resize((768, 512))
# --- END LOCAL IMAGE LOADING ---


In [ ]:
prompt = "A highly detailed character portrait blending **Rubeus Hagrid (Harry Potter)** and **Shrek (DreamWorks)**, inspired by the pose and likeness of the initial image. **Hagrid as an ogre**, featuring Shrek's iconic, friendly, and slightly gruff expression. They have warm, earthy green skin, subtle facial folds, and large, expressive eyes. **The character retains Hagrid's distinctive beard, long messy brown hair, and signature rugged clothing, but rendered in Shrek's aesthetic.** Emphasize a broad, round nose and prominent, short head-ears. The attire is rustic and worn, composed of coarse, natural fabrics, with subtle signs of swamp life like dirt or moss. The background is a blurred, atmospheric swamp with gnarled trees and dappled, magical light. **Cinematic lighting, photorealistic character design, intricate concept art, 8k, volumetric lighting, fantasy portrait.**"
neg = "deformed, ugly, disfigured, poor anatomy, extra limbs, missing limbs, bad proportions, distorted face, blurry, low quality, **low resolution**, **mutated**, cartoon, painting, illustration, **text, signature, watermark**"
images = pipe(prompt=prompt, image=init_image, negative_prompt=neg, ip_adapter_image=image,  strength=0.7, guidance_scale=7.5).images
images[0].save("fantasy_landscape.png")